# 인공 데이터

- 인위적으로 만든 데이터
    - 각 피처와 타깃값의 의미를 알 수 없음
    - 데이터에 아무런 의미를 부여하지 않음
    
    
- 모든 데이터는 범주형
    - 값이 두 개로만 구성된 이진 데이터
        - bin_
        
    - 순서형 데이터(ordinal data)
        - ord_
        - ord_3, ord_4, ord_5는 알파벳순서로 고유값 순서를 매김
        
    - 명목형 데이터(nominal data)
        - nom_
        
    - 날짜 데이터
        - day
        - month
        
- 종속변수도 이진 데이터
    - 0과 1로 구성됨

In [33]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate

# 데이터 확인

In [4]:
train_df = pd.read_csv("data/cat-in-the-dat/train.csv", index_col="id")
test_df = pd.read_csv("data/cat-in-the-dat/test.csv", index_col="id")
submission_df = pd.read_csv("data/cat-in-the-dat/sample_submission.csv", index_col="id")

In [5]:
train_df.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [6]:
test_df.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,Piano,...,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
300001,0,0,0,T,N,Red,Square,Lion,Canada,Piano,...,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
300002,1,0,1,F,Y,Blue,Square,Dog,China,Piano,...,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
300003,0,0,1,T,Y,Red,Star,Cat,China,Piano,...,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
300004,0,1,1,F,N,Red,Trapezoid,Dog,China,Piano,...,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [7]:
submission_df.head()

,target
id,
300000,0.5
300001,0.5
300002,0.5
300003,0.5
300004,0.5


In [8]:
train_df.describe(include="all")

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
count,300000.000000,300000.000000,300000.000000,300000,300000,300000,300000,300000,300000,300000,...,300000,300000.000000,300000,300000,300000,300000,300000,300000.000000,300000.000000,300000.00000
unique,NaN,NaN,NaN,2,2,3,6,6,6,4,...,11981,NaN,5,6,15,26,192,NaN,NaN,NaN
top,NaN,NaN,NaN,T,Y,Green,Trapezoid,Lion,Russia,Oboe,...,163cc60fa,NaN,Novice,Freezing,g,L,od,NaN,NaN,NaN
freq,NaN,NaN,NaN,153535,191633,127341,101181,101295,101123,92293,...,72,NaN,126583,99816,36405,19066,5019,NaN,NaN,NaN
mean,0.127503,0.256607,0.383223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.479567,NaN,NaN,NaN,NaN,NaN,3.007817,5.771113,0.30588
std,0.333537,0.436761,0.486173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.712524,NaN,NaN,NaN,NaN,NaN,1.817949,3.843519,0.46078
min,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,0.00000
25%,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,0.00000
50%,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,3.000000,4.000000,0.00000
75%,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,4.000000,9.000000,1.00000


- 모델 평가 기준 : AUC_SCORE

## 데이터 확인(컬럼별)

In [9]:
for col in train_df.columns:
    print(f"Columns : {col} : ", train_df[col].unique())

Columns : bin_0 :  [0 1]
Columns : bin_1 :  [0 1]
Columns : bin_2 :  [0 1]
Columns : bin_3 :  ['T' 'F']
Columns : bin_4 :  ['Y' 'N']
Columns : nom_0 :  ['Green' 'Blue' 'Red']
Columns : nom_1 :  ['Triangle' 'Trapezoid' 'Polygon' 'Square' 'Star' 'Circle']
Columns : nom_2 :  ['Snake' 'Hamster' 'Lion' 'Cat' 'Dog' 'Axolotl']
Columns : nom_3 :  ['Finland' 'Russia' 'Canada' 'Costa Rica' 'China' 'India']
Columns : nom_4 :  ['Bassoon' 'Piano' 'Theremin' 'Oboe']
Columns : nom_5 :  ['50f116bcf' 'b3b4d25d0' '3263bdce5' 'f12246592' '5b0f5acd5' '46cab09da'
 'be5592604' '72f8028dc' '4604905e7' 'ad95dc0ee' '2ff007c26' 'a35c346aa'
 'dbfb714a4' 'e1558b071' '39647c92a' 'ee55b9d67' '416a8f3ab' '91bde92fa'
 '3aa9329e2' '568550f04' 'caf83c0b5' '908a1b9c9' 'e844a1f66' 'dbc448931'
 '2979f0d45' '96c73114c' 'b7bb45938' '7da3e4aec' 'be4578201' 'dc07effb0'
 '88917a066' 'f2d59cf51' '5d18641ff' '9347491f2' '475e79160' 'e70a6270d'
 '5b1a9f841' 'e0efe9d20' 'd1b1709e8' '9bb7ea2da' '6fec43dd8' '0dee9b39a'
 '527ded12a' 

Columns : ord_5 :  ['kr' 'bF' 'Jc' 'kW' 'qP' 'PZ' 'wy' 'Ed' 'qo' 'CZ' 'qX' 'su' 'dP' 'aP'
 'MV' 'oC' 'RL' 'fh' 'gJ' 'Hj' 'TR' 'CL' 'Sc' 'eQ' 'kC' 'qK' 'dh' 'gM'
 'Jf' 'fO' 'Eg' 'KZ' 'Vx' 'Fo' 'sV' 'eb' 'YC' 'RG' 'Ye' 'qA' 'lL' 'Qh'
 'Bd' 'be' 'hT' 'lF' 'nX' 'kK' 'av' 'uS' 'Jt' 'PA' 'Er' 'Qb' 'od' 'ut'
 'Dx' 'Xi' 'on' 'Dc' 'sD' 'rZ' 'Uu' 'sn' 'yc' 'Gb' 'Kq' 'dQ' 'hp' 'kL'
 'je' 'CU' 'Fd' 'PQ' 'Bn' 'ex' 'hh' 'ac' 'rp' 'dE' 'oG' 'oK' 'cp' 'mm'
 'vK' 'ek' 'dO' 'XI' 'CM' 'Vf' 'aO' 'qv' 'jp' 'Zq' 'Qo' 'DN' 'TZ' 'ke'
 'cG' 'tP' 'ud' 'tv' 'aM' 'xy' 'lx' 'To' 'uy' 'ZS' 'vy' 'ZR' 'AP' 'GJ'
 'Wv' 'ri' 'qw' 'Xh' 'FI' 'nh' 'KR' 'dB' 'BE' 'Bb' 'mc' 'MC' 'tM' 'NV'
 'ih' 'IK' 'Ob' 'RP' 'dN' 'us' 'dZ' 'yN' 'Nf' 'QM' 'jV' 'sY' 'wu' 'SB'
 'UO' 'Mx' 'JX' 'Ry' 'Uk' 'uJ' 'LE' 'ps' 'kE' 'MO' 'kw' 'yY' 'zU' 'bJ'
 'Kf' 'ck' 'mb' 'Os' 'Ps' 'Ml' 'Ai' 'Wc' 'GD' 'll' 'aF' 'iT' 'cA' 'WE'
 'Gx' 'Nk' 'OR' 'Rm' 'BA' 'eG' 'cW' 'jS' 'DH' 'hL' 'Mf' 'Yb' 'Aj' 'oH'
 'Zc' 'qJ' 'eg' 'xP' 'vq' 'Id' 'pa' 'ux' 'kU' 'Cl']
Column

In [10]:
train_df.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1', 'nom_2',
       'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'ord_0',
       'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month', 'target'],
      dtype='object')

In [11]:
pd.get_dummies(train_df[["bin_0", "bin_1", "bin_2", "bin_3", "bin_4"]])

,bin_0,bin_1,bin_2,bin_3_F,bin_3_T,bin_4_N,bin_4_Y
id,,,,,,,
0,0,0,0,0,1,0,1
1,0,1,0,0,1,0,1
2,0,0,0,1,0,0,1
3,0,1,0,1,0,0,1
4,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...
299995,0,0,0,0,1,1,0
299996,0,0,0,1,0,0,1
299997,0,0,0,1,0,0,1


In [12]:
bin_0_df = pd.get_dummies(train_df["bin_0"], prefix="bin_0")

In [13]:
bin_1_df = pd.get_dummies(train_df["bin_1"], prefix="bin_1")

In [14]:
bin_2_df = pd.get_dummies(train_df["bin_2"], prefix="bin_2")

In [15]:
bin_3_to_4_df = pd.get_dummies(train_df[["bin_3", "bin_4"]])

In [16]:
nom_0_to_4_df = pd.get_dummies(train_df[["nom_0", "nom_1", "nom_2", "nom_3", "nom_4"]])

In [17]:
nom_5_df = pd.get_dummies(train_df["nom_5"])

In [18]:
train_df["nom_5"].map(lambda x: int(x, base=16))

id
0         21727636431
1         48239551952
2         13526359269
3         64728884626
4         24443727061
             ...     
299995    14485399999
299996    19108194839
299997     3739857818
299998    60487676017
299999    17560040363
Name: nom_5, Length: 300000, dtype: int64

In [19]:
nom_6_df = pd.get_dummies(train_df["nom_6"])

In [20]:
nom_7_df = pd.get_dummies(train_df["nom_7"])

In [22]:
nom_8_df = pd.get_dummies(train_df["nom_8"])

In [21]:
nom_9_df = pd.get_dummies(train_df["nom_9"])

In [23]:
ord_0_df = pd.get_dummies(train_df["ord_0"], prefix="ord_0")

In [24]:
ord_1_to_5_df = pd.get_dummies(train_df[["ord_1", "ord_2", "ord_3", "ord_4", "ord_5"]])

In [25]:
day_df = pd.get_dummies(train_df["day"], prefix="day")

In [26]:
month_df = pd.get_dummies(train_df["month"], prefix="month")

In [27]:
x = pd.concat([bin_0_df, bin_1_df, bin_2_df, bin_3_to_4_df, nom_0_to_4_df, nom_5_df, nom_6_df, nom_7_df, nom_8_df, nom_9_df, ord_0_df, ord_1_to_5_df, day_df, month_df], axis=1)

In [28]:
y = train_df["target"]

In [29]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300000 entries, 0 to 299999
Columns: 16461 entries, bin_0_0 to month_12
dtypes: uint8(16461)
memory usage: 4.6 GB


In [34]:
logi = LogisticRegression()

In [ ]:
# 성능 지표는 정확도(accuracy), 교차 검증 세트는 3개
scores = cross_val_score(logi, x, y, scoring='roc_auc', cv=5)
print('교차 검증별 정확도:', np.round(scores,4))
print('평균 검증 정확도:', np.round(np.mean(scores)))